In [1]:
import pandas as pd
import os
import time
import csv
import re
import codecs
import requests
from bs4 import BeautifulSoup
 
import datetime
    
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, \
        TimeoutException, ElementNotInteractableException, ElementClickInterceptedException

In [2]:
def text_file_to_dataframe(filename):
    # Check if the text file exists
    if os.path.exists(filename):
        
        # Read the text file and create a list of lines
        with open(filename, "r") as file:
            lines = file.readlines()
            content = [line.strip() for line in lines]

        # Create a DataFrame from the list
        df = pd.DataFrame(content, columns=['links'])
        
        return df
    else:
        print(f"{filename} does not exist.")
        return None

# Loading Links & Creating DataFrame

In [3]:
input_filename = 'final_links_again.txt'
df = text_file_to_dataframe(input_filename)

In [9]:
# new cell

df = df[15:1000]

In [10]:
# reset index to start at zero

df.reset_index(drop = True, inplace = True)
df.shape

(985, 11)

# Scraper Code

In [ ]:
# # S C R A P I N G   B R O W S E R
# driver = webdriver.Firefox()

In [ ]:
# df['links'].iloc[4]

In [ ]:
# driver.get('https://www.goodreads.com/choiceawards/best-fantasy-books-2022')

In [ ]:
# button = driver.find_element_by_css_selector('button[aria-label="Book details and editions"]')

In [ ]:
# button.click()

In [ ]:
# close_button = driver.find_element_by_class_name('gr-iconButton')
# close_button.text

In [ ]:
# close_button = driver.find_element_by_css_selector('.gr-iconButton')
# close_button.click()


In [ ]:
# <span class="Button__labelItem">Book details &amp; editions</span>

In [ ]:
import selenium

In [ ]:
selenium.__version__


In [6]:
# Define a function to extract the publisher element text
def get_publisher_text(publisher_element_WebDriverWait_amount):
    try:
        # Find the element with the EditionDetails class
        edition_details_element = WebDriverWait(driver, publisher_element_WebDriverWait_amount).until(EC.presence_of_element_located((By.CLASS_NAME, 'EditionDetails')))
        
        # Find the nested element with the data-testid attribute
        publisher_element = edition_details_element.find_element(By.CSS_SELECTOR, 'div[data-testid="contentContainer"]')
        
        return publisher_element[1].text

    except NoSuchElementException:
        print("Element: EditionDetails element after click fail")
        return ""
    except TimeoutException:
        print("Timeout: EditionDetails element not found within the specified time")
        return ""
    except ElementClickInterceptedException:
        print("N O   P U B L I S H E R")
        return ""

### This is the updated, v13

In [11]:
# S C R A P I N G   B R O W S E R
driver = webdriver.Chrome()

reviews_file_exists = os.path.isfile("remaining.csv")
index_file_exists = os.path.isfile("index.txt")

# checking last value of index in order to continue from last position
if index_file_exists:
    with open("index.txt", "r") as index_file:
        last_index = int(index_file.read())
else:
    last_index = 0

print(f"Output of last_index: {last_index}")
#============================================================================================================
# T I M E   W A I T   S E T T I N G S
#============================================================================================================
synopsis_and_review_wait_amount = 7
find_button_wait = 3
publisher_click_wait_amount = 7


button_element_WebDriverWait_amount = 20
title_element_WebDriverWait_amount = 15
synopsis_and_review_list_WebDriverWait_amount = 15
# publisher_element_WebDriverWait_amount = 3


# iterate over the dataframe
for index, row in df.iloc[last_index:].iterrows():
    #============================================================================================================
    # L O A D    P A G E    O N    S C R A P I N G    B R O W S E R
    #============================================================================================================
    driver.get(row['links'])
    
    #============================================================================================================
    # M E A T   A N D   P O T A T O E S
    #============================================================================================================
    try:
        print(f"- Waiting {synopsis_and_review_wait_amount} Seconds for Synopsis and Reviews to load....")
        time.sleep(synopsis_and_review_wait_amount)
        synopsis_and_review_list = WebDriverWait(driver, 
            synopsis_and_review_list_WebDriverWait_amount).until(EC.presence_of_all_elements_located(
            (By.CLASS_NAME, 'Formatted')))
        synopsis_length = len(synopsis_and_review_list)
    except TimeoutException:
        print(f"Timed out while waiting for synopsis and review list for book {title}, moving on to next book")
        continue
    #============================================================================================================
    # T I T L E
    #============================================================================================================
    try:
        title_element = WebDriverWait(driver, 
            title_element_WebDriverWait_amount).until(EC.presence_of_element_located((By.CLASS_NAME,
                                                        'BookPageTitleSection__title')))
        title = title_element.text
    except TimeoutException:
        print(f"Timed out while waiting for title for book {row['links']}, moving on to next book")
        continue

#     #============================================================================================================
#     # P U B L I S H E R
#     #============================================================================================================
#     # First, try to find and click the button
#     try:
#         print(f"Waiting {find_button_wait} seconds for button presense")
#         time.sleep(find_button_wait)
#         print(f"Soft Button Wait of {button_element_WebDriverWait_amount} seconds")
#         button = WebDriverWait(driver, button_element_WebDriverWait_amount).until(EC.presence_of_element_located(
#                                 (By.CSS_SELECTOR, 'button[aria-label="Book details and editions"]')))
        
#         find_button_test = "Pass"
#         #button = driver.find_element_by_css_selector('button[aria-label="Book details and editions"]')
#         button.click()
        
#         # if no exception from click() then assign "Pass"
#         clicked_button_test = "Pass"
        
#         # waiting for elements to load after clicking button
#         time.sleep(publisher_click_wait_amount)
#         print(f"- Waiting {publisher_click_wait_amount} seconds after clicking button")
        
#         # run the get_publisher_text function for 1st time
#         publisher = get_publisher_text(publisher_element_WebDriverWait_amount)
        
#     except (NoSuchElementException):
#         print("Button not found")
#         try:
#             publisher = get_publisher_text(publisher_element_WebDriverWait_amount)
#             find_button_test = "Pass"
#             clicked_button_test = "Pass"
#         except:
#             publisher=""
#             find_button_test = "Fail"
#             clicked_button_test = "Fail"
#     except (ElementNotInteractableException, ElementClickInterceptedException):
#         print("Button Not Clickable or Overlay is in the way\n")
#         try:
#             publisher = get_publisher_text(publisher_element_WebDriverWait_amount)
#             find_button_test = "Pass"
#             clicked_button_test = "Pass"
#         except:
#             publisher=""
#             find_button_test = "Fail"
#             clicked_button_test = "Fail"
#     #except:
#         #print("Unforeseen error: \n")
#         #publisher=""
#         #find_button_test = "Fails"
#         #clicked_button_test = "Fail"

#     # If the button is not found or not clickable, the get_publisher_text function will still run
#     #print("Everything Failed: One last try: \n")
#     #publisher = get_publisher_text()

    #============================================================================================================
    # D E F I N I T I O N    S E C T I O N
    #============================================================================================================
    try:
        page_count_element = driver.find_element(By.CSS_SELECTOR, 'p[data-testid="pagesFormat"]')
        page_count = page_count_element.text

    except NoSuchElementException:
        page_count = ""
    
    try:
        year_element = driver.find_element(By.CSS_SELECTOR, 'p[data-testid="publicationInfo"]')
        year = year_element.text
    except NoSuchElementException:
        year = ""
    
    try:
        review_count_element = driver.find_element(By.CSS_SELECTOR, 'span[data-testid="reviewsCount"]')
        review_count = review_count_element.text
        
        # converting string to interger with RegEx
        review_count_match = re.search(r'\d+', review_count)
        if review_count_match:
            review_count = int(review_count_match.group())
        else:
            review_count = 0
            
    except NoSuchElementException:
        review_count = 0

    try:
        rating_count_element = driver.find_element(By.CSS_SELECTOR, 'span[data-testid="ratingsCount"]')
        rating_count = rating_count_element.text

    except NoSuchElementException:
        rating_count = ""
    
    try:
        rating_element = driver.find_element(By.CLASS_NAME, 'RatingStatistics__rating')    
        rating = rating_element.text
        
    except NoSuchElementException:
        rating = ""
    
    try:
        genre_class_element = driver.find_element(By.CLASS_NAME, 'BookPageMetadataSection__genres')
        genres_text_element = genre_class_element.find_elements(By.CSS_SELECTOR, '.BookPageMetadataSection__genreButton .Button__labelItem')
        genre = genres_text_element[0].text
        
    except NoSuchElementException:
        genre = ""
    
    try:
        author_element = driver.find_element(By.CLASS_NAME, 'ContributorLink__name')
        author = author_element.text

    except NoSuchElementException:
        author = ""
    
    # assign the current datetime to this column for each row in the loop
    current_datetime = datetime.datetime.now()
    
    book_reviews = [title, page_count, year, review_count, rating, rating_count, genre, author, publisher, current_datetime]

    # current row in DataFrame. create these columns
    # and add book information to current dataframe row
    df.at[index, 'title'] = title
    df.at[index, 'page_count'] = page_count
    df.at[index, 'publisher'] = publisher
    df.at[index, 'year'] = year
    df.at[index, 'review_count'] = review_count
    df.at[index, 'rating'] = rating
    df.at[index, 'rating_count'] = rating_count
    df.at[index, 'genre'] = genre
    df.at[index, 'author'] = author
    df.at[index, 'scraped_at'] = current_datetime

    # ==================================================================
    # R E V I E W S
    # ==================================================================
    # iterating 5 times for reviews offset by 3 
    # list of elements "synopsis_and_review_list" has synopsis
    # and reviews
    
    # set this to the maximum number of reviews you want to capture
    max_reviews = 5  
    
    # index 0 has synopsis, index 1 & 2 are blank, 3-7 contain reviews
    review_index = 3 
    
    for i in range(max_reviews):
        if review_index < synopsis_length:
            # saving current review to temporary list
            book_reviews.append(synopsis_and_review_list[review_index].text)
            review_index += 1
        else:
            book_reviews.append("")
            break

    
    # ==================================================================
    # L O G
    # ==================================================================
    # T E S T   D E F I N I T I O N S
    # ------------------------------------------------------------------
    review_5_test = 'Pass' if synopsis_length >= 8 else 'Fail'
    publisher_test = 'Fail' if publisher == '' else 'Pass'
    print("===============================================================")
    print(f"            N E W   B O O K  S T A R T I N G:   #{index+1}")
    print("===============================================================")
    print("                      L I N K")
    print("---------------------------------------------------------------")
    print(row['links'])
    print("---------------------------------------------------------------")
    print(f"{title}")
    print("---------------------------------------------------------------")
    print(f"5 Review Test:{review_5_test}")
    print("---------------------------------------------------------------")
    print(f"Clicked Button Test:{clicked_button_test}")
    print("---------------------------------------------------------------")
    print(f"Publisher Found Test:{publisher_test}")
    print("---------------------------------------------------------------")
    print(f"TIME STAMP:{current_datetime}")
    print("---------------------------------------------------------------")
    # ==================================================================
    # S A V I N G   P L A C E   A F T E R   E A C H   I T E R A T I O N
    # ==================================================================
    with open("remaining.csv", "a", newline='', encoding='utf-8') as reviews_file:
        csv_writer = csv.writer(reviews_file)
        if not reviews_file_exists:
            csv_writer.writerow(['Book Name', 'page_count', 'year', 'review_count', 'rating', 'rating_count', 'genre', 'author', 'publisher', 'scraped_at', 'Review 1', 'Review 2', 'Review 3', 'Review 4', 'Review 5'])
            reviews_file_exists = True
        csv_writer.writerow(book_reviews)

    # ==================================================================
    # S A V I N G   P L A C E   A F T E R   E A C H   I T E R A T I O N
    # ==================================================================
    # save place of index to file for resuming later
    with open("index.txt", "w") as index_file:
        index_file.write(str(index + 1))

# D O N E    W I T H    S C R A P E R
driver.quit()

if not reviews_file_exists:
    open("remaining.csv", "a", newline='', encoding='utf-8').close()

if not index_file_exists:
    open("index.txt", "a").close()

Output of last_index: 0
- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #1
                      L I N K
---------------------------------------------------------------
https://www.goodreads.com/book/show/45725277-my-cousin-rachel-by-daphne-du-maurier-book-analysis?from_search=true&from_srp=true&qid=2EWuG4fnDY&rank=1
---------------------------------------------------------------
My Cousin Rachel by Daphne du Maurier (Book Analysis): Detailed Summary, Analysis and Reading Guide
---------------------------------------------------------------
5 Review Test:Fail
---------------------------------------------------------------
Clicked Button Test:Pass
---------------------------------------------------------------
Publisher Found Test:Fail
---------------------------------------------------------------
TIME STAMP:2023-03-23 12:18:45.151317
---------------------------------------------------------------
- Waiting 7 Seconds for Synopsis

- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #10
                      L I N K
---------------------------------------------------------------
https://www.goodreads.com/book/show/419346.The_View_from_Pompey_s_Head?from_search=true&from_srp=true&qid=YqSxilbEwU&rank=1
---------------------------------------------------------------
The View from Pompey's Head
---------------------------------------------------------------
5 Review Test:Pass
---------------------------------------------------------------
Clicked Button Test:Pass
---------------------------------------------------------------
Publisher Found Test:Fail
---------------------------------------------------------------
TIME STAMP:2023-03-23 12:20:03.339728
---------------------------------------------------------------
- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #11
                      L I N K
--------------

- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #19
                      L I N K
---------------------------------------------------------------
https://www.goodreads.com/book/show/526869.Peyton_Place?from_search=true&from_srp=true&qid=aJEXGsJItI&rank=1
---------------------------------------------------------------
Peyton Place #1
Peyton Place
---------------------------------------------------------------
5 Review Test:Pass
---------------------------------------------------------------
Clicked Button Test:Pass
---------------------------------------------------------------
Publisher Found Test:Fail
---------------------------------------------------------------
TIME STAMP:2023-03-23 12:21:20.247537
---------------------------------------------------------------
- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #20
                      L I N K
----------------------------

- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #28
                      L I N K
---------------------------------------------------------------
https://www.goodreads.com/book/show/5114.Raise_High_the_Roof_Beam_Carpenters_Seymour?from_search=true&from_srp=true&qid=PkCaeBuimT&rank=1
---------------------------------------------------------------
Raise High the Roof Beam, Carpenters & Seymour: An Introduction
---------------------------------------------------------------
5 Review Test:Pass
---------------------------------------------------------------
Clicked Button Test:Pass
---------------------------------------------------------------
Publisher Found Test:Fail
---------------------------------------------------------------
TIME STAMP:2023-03-23 12:22:36.242124
---------------------------------------------------------------
- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G: 

- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #37
                      L I N K
---------------------------------------------------------------
https://www.goodreads.com/book/show/20883294-valley-of-the-dolls-by-jacqueline-susann-lesson-plans?from_search=true&from_srp=true&qid=TpYlw2GX0M&rank=1
---------------------------------------------------------------
Valley of the Dolls by Jacqueline Susann Lesson Plans
---------------------------------------------------------------
5 Review Test:Fail
---------------------------------------------------------------
Clicked Button Test:Pass
---------------------------------------------------------------
Publisher Found Test:Fail
---------------------------------------------------------------
TIME STAMP:2023-03-23 12:23:56.388953
---------------------------------------------------------------
- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N

...

- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #945
                      L I N K
---------------------------------------------------------------
https://www.goodreads.com/book/show/49628.Cloud_Atlas
---------------------------------------------------------------
Cloud Atlas
---------------------------------------------------------------
5 Review Test:Pass
---------------------------------------------------------------
Clicked Button Test:Pass
---------------------------------------------------------------
Publisher Found Test:Fail
---------------------------------------------------------------
TIME STAMP:2023-03-23 14:35:00.590644
---------------------------------------------------------------
- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #946
                      L I N K
---------------------------------------------------------------
https://www.goodreads.com/book/sho

- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #954
                      L I N K
---------------------------------------------------------------
https://www.goodreads.com/book/show/49041.New_Moon
---------------------------------------------------------------
The Twilight Saga #2
New Moon
---------------------------------------------------------------
5 Review Test:Pass
---------------------------------------------------------------
Clicked Button Test:Pass
---------------------------------------------------------------
Publisher Found Test:Fail
---------------------------------------------------------------
TIME STAMP:2023-03-23 14:36:25.470973
---------------------------------------------------------------
- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #955
                      L I N K
---------------------------------------------------------------
https://www.goodrea

- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #963
                      L I N K
---------------------------------------------------------------
https://www.goodreads.com/book/show/3805.The_Corrections
---------------------------------------------------------------
The Corrections
---------------------------------------------------------------
5 Review Test:Pass
---------------------------------------------------------------
Clicked Button Test:Pass
---------------------------------------------------------------
Publisher Found Test:Fail
---------------------------------------------------------------
TIME STAMP:2023-03-23 14:37:52.847124
---------------------------------------------------------------
- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #964
                      L I N K
---------------------------------------------------------------
https://www.goodreads.com/b

- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #972
                      L I N K
---------------------------------------------------------------
https://www.goodreads.com/book/show/991197.The_Complete_Persepolis
---------------------------------------------------------------
Persepolis #1-4
The Complete Persepolis
---------------------------------------------------------------
5 Review Test:Pass
---------------------------------------------------------------
Clicked Button Test:Pass
---------------------------------------------------------------
Publisher Found Test:Fail
---------------------------------------------------------------
TIME STAMP:2023-03-23 14:39:18.038080
---------------------------------------------------------------
- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #973
                      L I N K
---------------------------------------------------------

- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #981
                      L I N K
---------------------------------------------------------------
https://www.goodreads.com/book/show/14201.Jonathan_Strange_Mr_Norrell
---------------------------------------------------------------
Strange & Norrell #1
Jonathan Strange & Mr Norrell
---------------------------------------------------------------
5 Review Test:Pass
---------------------------------------------------------------
Clicked Button Test:Pass
---------------------------------------------------------------
Publisher Found Test:Fail
---------------------------------------------------------------
TIME STAMP:2023-03-23 14:40:32.094490
---------------------------------------------------------------
- Waiting 7 Seconds for Synopsis and Reviews to load....
            N E W   B O O K  S T A R T I N G:   #982
                      L I N K
-------------------------------------------

### The above code, for me, only pulls author and reviews. Needing to trouble-shoot

### This is the old, v7. Got 730 books by running overnight (below code). Use the above version.

In [ ]:
# S C R A P I N G   B R O W S E R
driver = webdriver.Chrome()

reviews_file_exists = os.path.isfile("reviews.csv")
print("Does Review CSV File Exists?")
print(reviews_file_exists)
print("\n")

index_file_exists = os.path.isfile("index.txt")

# checking last value of index in order to continue from last position
if index_file_exists:
    with open("index.txt", "r") as index_file:
        last_index = int(index_file.read())
else:
    last_index = 0

print("Output of last_index")
print(last_index)

# iterate over the dataframe
for index, row in df.iloc[last_index:].iterrows():
    print("Ouput of index")
    print(index)
    
    print("Ouput of row")
    print(row)

    #============================================================================================================
    # L O A D    P A G E    O N    S C R A P I N G    B R O W S E R
    #============================================================================================================
    driver.get(row['links'])
    try:
        print("Waiting for Title Element to Load....")
        title_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'BookPageTitleSection__title')))
        print("Assigning Title")
        title = title_element.text
    except TimeoutException:
        print(f"Timed out while waiting for title for book {row['links']}, moving on to next book")
        continue
    
    print("Ouput of title")
    print(title)
    
    #============================================================================================================
    # P U B L I S H E R   D A T A
    #============================================================================================================
    # First, try to find and click the button
    button_found = False
    try:
        time.sleep(10)
        button = driver.find_element(By.CSS_SELECTOR, '//button[aria-label="Book details and editions"]')
        

        button.click()
        button_found = True
        # Wait for the publisher element to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "EditionDetails")))
    except (NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException):
        print("Button not found or not clickable")

    # If the button is not found or not clickable, the get_publisher_text function will still run
    publisher = get_publisher_text()

    # Print the extracted publisher text
    if publisher is not None:
        print("=====================")
        print(" P U B L I S H E R ")
        print("=====================")
        print(publisher)
        print("=====================")
    else:
        print("Publisher text not found")


    #============================================================================================================
    # D E F I N I T I O N    S E C T I O N
    #============================================================================================================
    try:
        print("Waiting for Synopsis and Reviews to load....")
        time.sleep(20)
        synopsis_and_review_list = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'Formatted')))
        print(f"The variable length of book {index+1} = ")
        print(len(synopsis_and_review_list))
        synopsis_length = len(synopsis_and_review_list)
        print("Output of synopsis_length")
        print(synopsis_length)
    except TimeoutException:
        print(f"Timed out while waiting for synopsis and review list for book {title}, moving on to next book")
        continue
    
    try:
        page_count_element = driver.find_element(By.XPATH, '//p[data-testid="pagesFormat"]')
        page_count = page_count_element.text  
        
        print("Ouput of page_count")
        print(page_count)
    except NoSuchElementException:
        page_count = ""
    
    try:
        year_element = driver.find_element(By.XPATH, '//p[data-testid="publicationInfo"]')
        year = year_element.text
        
        print("Ouput of year")
        print(year)
    except NoSuchElementException:
        year = ""
    
    try:
        review_count_element = driver.find_element(By.XPATH, '//span[data-testid="reviewsCount"]')
        review_count = review_count_element.text
        
        print("Ouput of review_count")
        print(review_count)
        
        # converting string to interger with RegEx
        review_count_match = re.search(r'\d+', review_count)
        if review_count_match:
            review_count = int(review_count_match.group())
            print("Ouput of RegEx: review_count")
            print(review_count)
        else:
            review_count = 0
            print("Ouput of No_RegEx: review_count")
            print(review_count)
            
    except NoSuchElementException:
        review_count = 0
        print("Ouput of No_Count: review_count")
        print(review_count)
    
    try:
        rating_count_element = driver.find_element(By.XPATH, '//span[data-testid="ratingsCount"]')
        rating_count = rating_count_element.text 
        print("Ouput of rating_count")
        print(rating_count)
    except NoSuchElementException:
        rating_count = ""
    
    try:
        rating_element = driver.find_element(By.XPATH, '//RatingStatistics__rating')
        rating = rating_element.text
        print("Ouput of rating")
        print(rating)
        
    except NoSuchElementException:
        rating = ""
    
    try:
        genre_class_element = driver.find_element(By.XPATH, '//BookPageMetadataSection__genres')
        genres_text_element = genre_class_element.find_elements(By.XPATH, '//.BookPageMetadataSection__genreButton .Button__labelItem')
        genre = genres_text_element[0].text
        print("Ouput of genre")
        print(genre)
        
    except NoSuchElementException:
        genre = ""
    
    try:
        author_element = driver.find_element(By.XPATH, '//ContributorLink__name')('ContributorLink__name')
        author = author_element.text
        print("Ouput of author")
        print(author)

    except NoSuchElementException:
        author = ""
        
    book_reviews = [title, page_count, year, review_count, rating, rating_count, genre, author,publisher]
    
    # current row in DataFrame. create these columns
    # and add book information to current dataframe row
    df.at[index, 'page_count'] = page_count
    df.at[index, 'publisher'] = publisher
    df.at[index, 'year'] = year
    df.at[index, 'review_count'] = review_count
    df.at[index, 'rating'] = rating
    df.at[index, 'rating_count'] = rating_count
    df.at[index, 'genre'] = genre
    df.at[index, 'author'] = author

    
    # =====================
    # T E S T   P R I N T
    # =====================
    print(f"Book {index}:")
    print(title)
    print("==============")
    print(page_count)
    print("\n")
    print("==============")
    print(year)
    print("\n")
    print("==============")
    print("Total Reviews: ")
    print(review_count)
    print("\n")
    print("==============")
    print(rating)
    print("\n")
    print("==============")
    print(rating_count)
    print("\n")
    print("==============")
    print(genre)
    print("\n")
    print("==============")
    print(author)
    print("\n")
    print("==============")
    print(publisher)
    print("\n")
    # ==================================================================
    # R E V I E W S
    # ==================================================================
    # iterating 5 times for reviews offset by 3 
    # list of elements "synopsis_and_review_list" has synopsis
    # and reviews
    review_index = 3
    for i in range(5):
        if review_index < synopsis_length:
            print(f"Review {i + 1}")
            print("==============")
            print(synopsis_and_review_list[review_index].text)

            # saving current review to temporary list
            book_reviews.append(synopsis_and_review_list[review_index].text)
            review_index += 1
        else:
            book_reviews.append("")
    # ==================================================================
    # S A V I N G   P L A C E   A F T E R   E A C H   I T E R A T I O N
    # ==================================================================
    with open("reviews.csv", "a", newline='', encoding='utf-8') as reviews_file:
        csv_writer = csv.writer(reviews_file)
        if not reviews_file_exists:
            csv_writer.writerow(['Book Name', 'page_count', 'year', 'review_count', 'rating', 'rating_count', 'genre', 'author', 'publisher', 'Review 1', 'Review 2', 'Review 3', 'Review 4', 'Review 5'])
            reviews_file_exists = True
        csv_writer.writerow(book_reviews)

    # ==================================================================
    # S A V I N G   P L A C E   A F T E R   E A C H   I T E R A T I O N
    # ==================================================================
    # save place of index to file for resuming later
    with open("index.txt", "w") as index_file:
        index_file.write(str(index + 1))

# D O N E    W I T H    S C R A P E R
driver.quit()

if not reviews_file_exists:
    open("reviews.csv", "a", newline='', encoding='utf-8').close()

if not index_file_exists:
    open("index.txt", "a").close()